In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# set some constants



In [8]:
num_classes = 5  # Number of cat breeds
image_size = (249, 249)
batch_size = 5
num_epochs = 10

train_path = '/content/drive/MyDrive/Colab Notebooks/sws/baseline/Cats'
test_path = '/content/drive/MyDrive/Colab Notebooks/sws/baseline/Cats'

# set model parameters

In [9]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import os



MODEL_NAME = "cats.hd5"

def build_model(model_name):
  if os.path.exists(model_name):
    print("\n using the exising model : "+model_name)
    model = load_model(model_name)
  else:
    # Constants
    num_classes = 5  # Number of cat breeds
    image_size = (249, 249)
    batch_size = 5
    num_epochs = 10

    # Load the VGG16 model without the top classification layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification layers on top of the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Create the final model
    model = Model(inputs=base_model.input, outputs=predictions)



  return model


# set data preprocessing parameters


In [10]:
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
import numpy as np
from PIL import Image
from os import listdir
from os.path import join


dict = {0:'persian_cats', 1:'Ragdoll', 2:'Scottish fold cats', 3:'Singapura', 4:'Sphynx cat'}

session = tf.compat.v1.Session(graph=tf.compat.v1.Graph())
def classify(model, image):
  with session.graph.as_default():
    set_session(session)
    result = model.predict(image)
    themax = np.argmax(result)

  return (dict[themax], result[0][themax], themax)


# change the image into the size we want
def load_image(image_fname):
  img = Image.open(image_fname)
  img = img.resize(249, 249)
  imgarray = np.array(img)/255.0
  final = np.expan_dims(imgarray, axis=0)
  return final

# Prepare the data augmentation generator
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_data_generator = ImageDataGenerator(rescale=1./255)



# Load the training and testing datasets
train_generator = train_data_generator.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_data_generator.flow_from_directory(
    test_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 907 images belonging to 5 classes.
Found 907 images belonging to 5 classes.


# set training function

In [11]:
def train(model, epochs, model_name, batch_size):
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  savemodel = ModelCheckpoint(model_name)
  stopmodel = EarlyStopping(min_delta=0.001, patience=10)

  print("start training")

  model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[savemodel, stopmodel]
  )


# train the model

In [ ]:
# Train the model


epoch = 2
model = build_model(MODEL_NAME)
train(model, epoch, MODEL_NAME, batch_size)




start training
Epoch 1/2
 49/181 [=======>......................] - ETA: 9:34 - loss: 1.4862 - accuracy: 0.4122